
### The University of Melbourne, School of Computing and Information Systems
# COMP30027 Machine Learning, 2020 Semester 1

## Assignment 1: Naive Bayes Classifiers

###### Submission deadline: 7 pm, Monday 20 Apr 2020

**Student Name(s):**    Shuyang Fan, Yiran Wang 

**Student ID(s):**     988301, 987751


This iPython notebook is a template which you will use for your Assignment 1 submission.

Marking will be applied on the four functions that are defined in this notebook, and to your responses to the questions at the end of this notebook (Submitted in a separate PDF file).

**NOTE: YOU SHOULD ADD YOUR RESULTS, DIAGRAMS AND IMAGES FROM YOUR OBSERVATIONS IN THIS FILE TO YOUR REPORT (the PDF file).**

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find.

**Adding proper comments to your code is MANDATORY. **

In [62]:
import pandas as pd
import math
import numpy as np
from pandas.api.types import is_numeric_dtype

In [6]:
def read_data(fileName):
    data = pd.read_csv(fileName, header=None)
    X = data.iloc[:,:-1]
    Y = data.iloc[:,-1]
    # print(X.head(5))
    # print(Y.head(5))
    return X, Y


# Train Test Split

In [10]:
(X, Y) = read_data("./datasets/adult.data")

def train_test_split(X, y, test_size=0.2):
    X_total = X.shape[0]
    assert(X_total == Y.size)
    arr_rand = np.random.rand(X.shape[0])
    split = arr_rand < np.percentile(arr_rand, test_size*100)
    X_train = X[split]
    y_train = y[split]
    X_test =  X[~split]
    y_test = y[~split]
    return X_train, X_test, y_train, y_test

(X_train, X_test, y_train, y_test) = train_test_split(X,Y)

In [63]:
def preprocess():
    print("Hello World")
    return

# This function should prepare the data by reading it from a file and converting it into a useful format for training and testing
def discretizer(X, column_index, bin_size):
    for index, column in enumerate(column_index):
        X.iloc[:,column] = pd.cut(X.iloc[:,column], bin_size[index])

(X, Y) = read_data("./datasets/adult.data")       


numeric_column = []

for column in range(X.shape[1]):
    if (is_numeric_dtype(X.iloc[:,column])):
        numeric_column.append(column)
        
discretizer(X, numeric_column, [10 for i in range(len(numeric_column))])
X.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,"(38.9, 46.2]",State-gov,"(10812.58, 159527.0]",Bachelors,"(11.5, 13.0]",Never-married,Adm-clerical,Not-in-family,White,Male,"(-99.999, 9999.9]","(-4.356, 435.6]","(30.4, 40.2]",United-States
1,"(46.2, 53.5]",Self-emp-not-inc,"(10812.58, 159527.0]",Bachelors,"(11.5, 13.0]",Married-civ-spouse,Exec-managerial,Husband,White,Male,"(-99.999, 9999.9]","(-4.356, 435.6]","(10.8, 20.6]",United-States
2,"(31.6, 38.9]",Private,"(159527.0, 306769.0]",HS-grad,"(8.5, 10.0]",Divorced,Handlers-cleaners,Not-in-family,White,Male,"(-99.999, 9999.9]","(-4.356, 435.6]","(30.4, 40.2]",United-States
3,"(46.2, 53.5]",Private,"(159527.0, 306769.0]",11th,"(5.5, 7.0]",Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,"(-99.999, 9999.9]","(-4.356, 435.6]","(30.4, 40.2]",United-States
4,"(24.3, 31.6]",Private,"(306769.0, 454011.0]",Bachelors,"(11.5, 13.0]",Married-civ-spouse,Prof-specialty,Wife,Black,Female,"(-99.999, 9999.9]","(-4.356, 435.6]","(30.4, 40.2]",Cuba


In [12]:
def get_prior(y_train):
    train_inputs = y_train
    labels, counts = np.unique(train_inputs, return_counts=True)
    prior = {}
    for i, label in enumerate(labels):
        prior[label] = float(counts[i])/len(train_inputs)
    return prior      

X_train, X_test, y_train, y_test = train_test_split(X,Y)
get_prior(y_train)

{'<=50K': 0.7590601965601965, '>50K': 0.24093980343980345}

In [13]:
# This function should calculat prior probabilities and likelihoods from the training data and using
# them to build a naive Bayes model

def train(X_train, y_train):
    grouped = X_train.groupby(y_train)
    for label in np.unique(y_train):
        print(grouped.get_group(label))
    return

X_train, X_test, y_train, y_test = train_test_split(X,Y)
train(X_train, y_train)

       0                 1       2             3   4                   5   \
1      50  Self-emp-not-inc   83311     Bachelors  13  Married-civ-spouse   
21     54           Private  302146       HS-grad   9           Separated   
26     19           Private  168294       HS-grad   9       Never-married   
31     20           Private  266015  Some-college  10       Never-married   
33     30       Federal-gov   59951  Some-college  10  Married-civ-spouse   
40     31           Private  507875           9th   5  Married-civ-spouse   
43     49           Private   94638       HS-grad   9           Separated   
54     47      Self-emp-inc  109832       HS-grad   9            Divorced   
66     28           Private  183175  Some-college  10            Divorced   
79     31         Local-gov  125927       7th-8th   4  Married-civ-spouse   
88     30           Private   54334           9th   5       Never-married   
92     28           Private  212563  Some-college  10            Divorced   

In [14]:
# This function should predict classes for new items in a test dataset (for the purposes of this assignment, you
# can re-use the training data as a test set)

def predict():
    return

In [15]:
# This function should evaliate the prediction performance by comparing your model’s class outputs to ground
# truth labels

def evaluate():
    return

## Questions 


If you are in a group of 1, you will respond to question (1), and **one** other of your choosing (two responses in total).

If you are in a group of 2, you will respond to question (1) and question (2), and **two** others of your choosing (four responses in total). 

A response to a question should take about 100–250 words, and make reference to the data wherever possible.

#### NOTE: you may develope codes or functions in respond to the question, but your formal answer should be added to a separate file.

### Q1
Try discretising the numeric attributes in these datasets and treating them as discrete variables in the na¨ıve Bayes classifier. You can use a discretisation method of your choice and group the numeric values into any number of levels (but around 3 to 5 levels would probably be a good starting point). Does discretizing the variables improve classification performance, compared to the Gaussian na¨ıve Bayes approach? Why or why not?

### Q2
Implement a baseline model (e.g., random or 0R) and compare the performance of the na¨ıve Bayes classifier to this baseline on multiple datasets. Discuss why the baseline performance varies across datasets, and to what extent the na¨ıve Bayes classifier improves on the baseline performance.

### Q3
Since it’s difficult to model the probabilities of ordinal data, ordinal attributes are often treated as either nominal variables or numeric variables. Compare these strategies on the ordinal datasets provided. Deterimine which approach gives higher classification accuracy and discuss why.

### Q4
Evaluating the model on the same data that we use to train the model is considered to be a major mistake in Machine Learning. Implement a hold–out or cross–validation evaluation strategy (you should implement this yourself and do not simply call existing implementations from `scikit-learn`). How does your estimate of effectiveness change, compared to testing on the training data? Explain why. (The result might surprise you!)

### Q5
Implement one of the advanced smoothing regimes (add-k, Good-Turing). Does changing the smoothing regime (or indeed, not smoothing at all) affect the effectiveness of the na¨ıve Bayes classifier? Explain why, or why not.

### Q6
The Gaussian na¨ıve Bayes classifier assumes that numeric attributes come from a Gaussian distribution. Is this assumption always true for the numeric attributes in these datasets? Identify some cases where the Gaussian assumption is violated and describe any evidence (or lack thereof) that this has some effect on the NB classifier’s predictions.